In [1]:
import pandas as pd
import numpy as np

In [4]:
distribution_centers = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/distribution_centers.csv')
event = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/events.csv')
inventory_items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/inventory_items.csv')
order_items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/order_items.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/products.csv')
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종 프로젝트/루커스 데이터셋/users.csv')


# order_items 파악
status 값 : 'Cancelled', 'Complete', 'Shipped', 'Processing', 'Returned'


1.   테이블 전체 케이스 : 181759
2. processing의 order_id 수 : 25156개
3. complete의 order_id 수 : 31354개
4. shipped의 order_id 수 : 54440개
5. returned의 order_id 수 : 12530개
6. cancelled의 order_id 수 : 18609개








In [12]:
#각 국가별로 status 비중이 어떻게 되는지 파악
#본 셀 아래에 있는 셀들부터 실행할 것!!!

dfs = [whole_users_by_country, complete_users_by_country, processing_users_by_country, shipped_users_by_country, returned_users_by_country, cancelled_users_by_country]

#한번에 여러 테이블 병합하기
merged_df = dfs[0]   #첫번째 테이블(whole_users_by_country) 기준, 그 다음 테이블부터 순차적으로 병합
for df in dfs[1:]:
  merged_df = pd.merge(merged_df, df, on = 'country')


#전체 대비 비중 구하기
#문자열인 country 칼럼 일단 제외
country_column = merged_df['country']
#country칼럼 제외한 나머지 칼럼들을 whole로 나눠주고 100곱해서 비중 구하기. 마지막에 소수 둘째자리에서 반올림
merged_df = (merged_df.iloc[:, 1:].div(merged_df['n_of_whole'], axis= 0)*100).round(2)
#insert함수로 첫번째 칼럼(0)으로 삽입. 이름도 다시 country
merged_df.insert(0, 'country', country_column)
merged_df

,country,n_of_whole,n_of_complete,n_of_processing,n_of_shipped,n_of_returned,n_of_cancelled
0,Australia,100.0,25.21,20.95,28.97,10.34,14.52
1,Belgium,100.0,24.22,19.92,32.76,10.13,12.98
2,Brasil,100.0,25.25,20.12,29.35,10.03,15.24
3,China,100.0,24.92,19.86,30.02,10.03,15.17
4,Colombia,100.0,31.58,21.05,21.05,10.53,15.79
5,France,100.0,24.46,19.47,30.85,10.29,14.93
6,Germany,100.0,24.95,20.77,30.29,9.21,14.77
7,Japan,100.0,25.91,19.49,30.05,10.49,14.06
8,Poland,100.0,24.00,23.69,28.00,10.77,13.54
9,South Korea,100.0,25.05,19.79,30.71,10.41,14.05


In [5]:
#user에서 필요한 칼럼만 가져오기
user_temp = users.sort_values(by = 'id')[['id', 'country', 'state', 'city', 'street_address', 'latitude', 'longitude']]

In [6]:
whole_t = pd.DataFrame(order_items.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()  #중간에 집계값 안 구하면 order_id카운트가 중복되게 되어버림;;
#processing_t로 전체 order_id 수 볼 수 있음

whole_temp = whole_t.groupby('user_id')['order_id'].count().reset_index()

#whole인 user의 주소 정보 확인 테이블 만들기
whole_user_info = pd.merge(whole_temp, user_temp, left_on='user_id', right_on='id', how = 'left')


#country 부분만 한 번에 수정하면 집계 기준 바꿀 수 있음
whole_users_by_country = whole_user_info.groupby(['country'])['order_id'].sum().reset_index().rename(columns = {'order_id' : 'n_of_whole'})
whole_users_by_country

,country,n_of_whole
0,Australia,2630
1,Austria,2
2,Belgium,1441
3,Brasil,18262
4,China,42986
5,Colombia,19
6,Deutschland,1
7,España,4
8,France,5968
9,Germany,5286


In [7]:
#processing 이면 shipped_at도 delivered_at도 returned_at도 없음
#status가 processing인 것을 추출한 것을 기반, user별 order_id수 카운트하기
processing = order_items.query('status == "Processing"').reset_index()
processing_t = pd.DataFrame(processing.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()  #중간에 집계값 안 구하면 order_id카운트가 중복되게 되어버림;;
#processing_t로 전체 order_id 수 볼 수 있음

processing_temp = processing_t.groupby('user_id')['order_id'].count().reset_index()

#processing인 user의 주소 정보 확인 테이블 만들기
processing_user_info = pd.merge(processing_temp, user_temp, left_on='user_id', right_on='id', how = 'left')


#country 부분만 한 번에 수정하면 집계 기준 바꿀 수 있음
processing_users_by_country = processing_user_info.groupby(['country'])['order_id'].sum().reset_index().rename(columns = {'order_id' : 'n_of_processing'})
processing_users_by_country

,country,n_of_processing
0,Australia,551
1,Belgium,287
2,Brasil,3675
3,China,8535
4,Colombia,4
5,España,2
6,France,1162
7,Germany,1098
8,Japan,574
9,Poland,77


In [8]:
#complete
#status가 complete인 것을 추출한 것을 기반, user별 order_id수 카운트하기
complete = order_items.query('status == "Complete"').reset_index()
complete_t = pd.DataFrame(complete.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()  #중간에 집계값 안 구하면 order_id카운트가 중복되게 되어버림;;

complete_temp = complete_t.groupby('user_id')['order_id'].count().reset_index()


#complete인 user의 주소 정보 확인 테이블 만들기
complete_user_info = pd.merge(complete_temp, user_temp, left_on='user_id', right_on='id', how = 'left')

#country 부분만 한 번에 수정하면 집계 기준 바꿀 수 있음
complete_users_by_country = complete_user_info.groupby(['country'])['order_id'].sum().reset_index().rename(columns = {'order_id' : 'n_of_complete'})
complete_users_by_country

,country,n_of_complete
0,Australia,663
1,Austria,1
2,Belgium,349
3,Brasil,4612
4,China,10713
5,Colombia,6
6,Deutschland,1
7,España,1
8,France,1460
9,Germany,1319


In [9]:
#shipped
#status가 shipped인 것을 추출한 것을 기반, user별 order_id수 카운트하기
shipped = order_items.query('status == "Shipped"').reset_index()
shipped_t = pd.DataFrame(shipped.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()  #중간에 집계값 안 구하면 order_id카운트가 중복되게 되어버림;;

shipped_temp = shipped_t.groupby('user_id')['order_id'].count().reset_index()


#complete인 user의 주소 정보 확인 테이블 만들기
shipped_user_info = pd.merge(shipped_temp, user_temp, left_on='user_id', right_on='id', how = 'left')

#country 부분만 한 번에 수정하면 집계 기준 바꿀 수 있음
shipped_users_by_country = shipped_user_info.groupby(['country'])['order_id'].sum().reset_index().rename(columns = {'order_id' : 'n_of_shipped'})
shipped_users_by_country

,country,n_of_shipped
0,Australia,762
1,Austria,1
2,Belgium,472
3,Brasil,5360
4,China,12905
5,Colombia,4
6,France,1841
7,Germany,1601
8,Japan,885
9,Poland,91


In [10]:
#returned
#status가 returned인 것을 추출한 것을 기반, user별 order_id수 카운트하기
returned = order_items.query('status == "Returned"').reset_index()
returned_t = pd.DataFrame(returned.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()  #중간에 집계값 안 구하면 order_id카운트가 중복되게 되어버림;;

returned_temp = returned_t.groupby('user_id')['order_id'].count().reset_index()


#complete인 user의 주소 정보 확인 테이블 만들기
returned_user_info = pd.merge(returned_temp, user_temp, left_on='user_id', right_on='id', how = 'left')

#country 부분만 한 번에 수정하면 집계 기준 바꿀 수 있음
returned_users_by_country = returned_user_info.groupby(['country'])['order_id'].sum().reset_index().rename(columns = {'order_id' : 'n_of_returned'})
returned_users_by_country

,country,n_of_returned
0,Australia,272
1,Belgium,146
2,Brasil,1831
3,China,4313
4,Colombia,2
5,España,1
6,France,614
7,Germany,487
8,Japan,309
9,Poland,35


In [11]:
#Cancelled
#status가 cancelled인 것을 추출한 것을 기반, user별 order_id수 카운트하기
cancelled = order_items.query('status == "Cancelled"').reset_index()
cancelled_t = pd.DataFrame(cancelled.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()  #중간에 집계값 안 구하면 order_id카운트가 중복되게 되어버림;;

cancelled_temp = cancelled_t.groupby('user_id')['order_id'].count().reset_index()


#complete인 user의 주소 정보 확인 테이블 만들기
cancelled_user_info = pd.merge(cancelled_temp, user_temp, left_on='user_id', right_on='id', how = 'left')

#country 부분만 한 번에 수정하면 집계 기준 바꿀 수 있음
cancelled_users_by_country = cancelled_user_info.groupby(['country'])['order_id'].sum().reset_index().rename(columns = {'order_id' : 'n_of_cancelled'})
cancelled_users_by_country

,country,n_of_cancelled
0,Australia,382
1,Belgium,187
2,Brasil,2784
3,China,6520
4,Colombia,3
5,France,891
6,Germany,781
7,Japan,414
8,Poland,44
9,South Korea,930


# 이 아래는 별 의미X

In [ ]:
import folium

def create_map(locations):
    # 지도 객체 생성
    map_object = folium.Map(location=[locations[0][0], locations[0][1]], zoom_start=10)

    # 각 위치에 대해 마커 추가
    for loc in locations:
        folium.Marker([loc[0], loc[1]]).add_to(map_object)

    return map_object


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rows_as_lists = []
for index, row in distribution_centers[['latitude', 'longitude']].iterrows():
    rows_as_lists.append(row.tolist())

print(rows_as_lists)

[[35.1174, -89.9711], [41.8369, -87.6847], [29.7604, -95.3698], [34.05, -118.25], [29.95, -90.0667], [40.634, -73.7834], [39.95, -75.1667], [30.6944, -88.0431], [32.7833, -79.9333], [32.0167, -81.1167]]


In [ ]:
locations = rows_as_lists
create_map(locations)


In [ ]:
event['created_at'] = pd.to_datetime(event['created_at'])
event['time'] = event['created_at'].dt.date

In [ ]:
event['year_month'] = event['time'].apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
order_items['status'].unique()

array(['Cancelled', 'Complete', 'Shipped', 'Processing', 'Returned'],
      dtype=object)

In [ ]:
temp = pd.DataFrame(order_items.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()
temp.head(10)

,user_id,order_id,sale_price
0,1,1,159.990000
1,2,2,22.000000
2,3,3,59.990002
3,3,4,108.750000
4,3,5,109.989998
5,3,6,123.470001
6,4,7,148.000000
7,5,8,65.650000
8,6,9,144.820004
9,7,10,84.209999


In [ ]:
temp.groupby('user_id')['sale_price'].sum()

user_id
1         159.990000
2          22.000000
3         402.200001
4         148.000000
5          65.650000
             ...    
99993      26.000000
99996     123.870001
99998     116.990000
99999      67.980000
100000    152.449997
Name: sale_price, Length: 80044, dtype: float64

In [ ]:
n_of_orders_by_users = pd.DataFrame(temp.groupby('user_id')['order_id'].count()).reset_index().rename(columns= {'order_id' : 'order_counts'})
user_total_order_price = pd.merge(n_of_orders_by_users, temp.groupby('user_id')['sale_price'].sum(), on = 'user_id')  #캔슬하고 리턴 감안 안 한 것

In [ ]:
user_total_order_price.sort_values(by = 'sale_price', ascending = False)

,user_id,order_counts,sale_price
65038,81262,2,1738.969982
36897,46234,3,1576.699999
75381,94183,4,1570.990002
43396,54272,2,1490.000000
12567,15746,4,1488.799999
...,...,...,...
12969,16244,1,1.510000
33395,41873,1,1.500000
55934,69897,1,1.500000
30604,38312,1,0.490000


In [ ]:
#취소 및 반품 안 한 것들 집계
t2 = order_items.query('status == "Complete" or status == "Processing" or status == "Shipped"')
t2 = pd.DataFrame(t2.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()
user_total_order_net_price = pd.merge(t2.groupby('user_id')['order_id'].count(), t2.groupby(['user_id'])['sale_price'].sum(), on = 'user_id').reset_index().rename(columns = {'order_id' : 'order_counts'})



#취소 및 반품한 주문 수 및 취소금액 집계, 그리고 칼럼 이름 바꾸기
t3 = order_items.query('status == "Cancelled" or status == "Returned"')
t3 = pd.DataFrame(t3.groupby(['user_id', 'order_id'])['sale_price'].sum()).reset_index()
user_total_order_cancelled_price = pd.merge(t3.groupby('user_id')['order_id'].count(), t3.groupby(['user_id'])['sale_price'].sum(), on = 'user_id').reset_index().rename(columns = {'order_id' : 'order_counts'})
user_total_order_cancelled_price = user_total_order_cancelled_price.rename(columns = {'order_counts' : 'cancel_counts', 'sale_price' : 'cancelled_price'})


orders_and_cancelation_per_each_customer = pd.merge(user_total_order_net_price, user_total_order_cancelled_price, on = 'user_id', how = 'left')
orders_and_cancelation_per_each_customer

,user_id,order_counts,sale_price,cancel_counts,cancelled_price
0,1,1,159.990000,NaN,NaN
1,2,1,22.000000,NaN,NaN
2,3,4,402.200001,NaN,NaN
3,4,1,148.000000,NaN,NaN
4,6,1,144.820004,NaN,NaN
...,...,...,...,...,...
66249,99992,1,55.950001,NaN,NaN
66250,99996,2,123.870001,NaN,NaN
66251,99998,3,116.990000,NaN,NaN
66252,99999,1,67.980000,NaN,NaN


In [ ]:
user_total_order_net_price


,user_id,order_counts,sale_price
0,1,1,159.990000
1,2,1,22.000000
2,3,4,402.200001
3,4,1,148.000000
4,6,1,144.820004
...,...,...,...
66249,99992,1,55.950001
66250,99996,2,123.870001
66251,99998,3,116.990000
66252,99999,1,67.980000


In [ ]:
order_items.query('status == "Complete" or status == "Processing" or status == "Shipped"')['user_id'].nunique()

66254

In [ ]:

user_total_order_cancelled_price

,user_id,cancel_counts,cancelled_price
0,5,1,65.650000
1,8,1,80.689999
2,9,2,543.489986
3,11,1,132.000000
4,12,1,27.980000
...,...,...,...
27486,99982,1,19.990000
27487,99988,1,68.000000
27488,99990,1,199.000000
27489,99993,1,26.000000


In [ ]:
users['id'].nunique()

100000

In [ ]:
orders['status'].unique()

array(['Cancelled', 'Complete', 'Processing', 'Returned', 'Shipped'],
      dtype=object)

In [ ]:
event.query('event_type=="purchase"')

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type,time,year_month
23067,808414,61899.0,13,1daacb69-a4b6-4ef6-8686-bc62f86e87d5,2023-02-25 01:33:38+00:00,191.109.103.15,São Paulo,São Paulo,02220-000,Chrome,Facebook,/purchase,purchase,2023-02-25,2023-02
23068,984631,75239.0,10,43d77105-0da0-4381-b9da-edc7e7ae0045,2022-10-01 23:44:19+00:00,52.83.112.123,São Paulo,São Paulo,02675-031,Chrome,Email,/purchase,purchase,2022-10-01,2022-10
23069,121732,9453.0,7,aaee2565-3d4a-4b8c-8e46-94e3279aabc8,2023-08-22 08:12:02+00:00,220.118.43.166,São Paulo,São Paulo,02675-031,Chrome,Adwords,/purchase,purchase,2023-08-22,2023-08
23070,1071846,81970.0,10,d0f6607c-3f4f-476f-9153-81793c782ebd,2020-11-01 10:01:37+00:00,158.127.181.83,São Paulo,São Paulo,02675-031,Safari,Adwords,/purchase,purchase,2020-11-01,2020-11
23071,1144499,87587.0,13,5ce2b5d7-5c19-407e-bbf0-af2803d7295c,2021-06-19 14:36:46+00:00,7.61.124.107,São Paulo,São Paulo,02675-031,Other,Email,/purchase,purchase,2021-06-19,2021-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431958,822590,62906.0,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase,2022-04-07,2022-04
2431959,43178,3333.0,5,e472549e-273e-402e-ae5f-38ece25699c2,2023-09-26 10:25:28+00:00,97.78.106.249,Orrell,England,WN5,IE,Email,/purchase,purchase,2023-09-26,2023-09
2431960,553230,42386.0,10,6a7ba027-2af2-46a3-ae05-294a264840cf,2023-07-31 15:11:10+00:00,147.252.137.75,Orrell,England,WN5,Chrome,Facebook,/purchase,purchase,2023-07-31,2023-07
2431961,1015190,77546.0,5,29e5960c-214b-456e-8c38-fe6eb8ff1ec8,2024-01-02 14:26:38+00:00,112.209.120.82,Wolverhampton,England,WV6,Chrome,Adwords,/purchase,purchase,2024-01-02,2024-01


In [ ]:
event

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type,time,year_month
0,2198523,NaN,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel,2021-06-17,2021-06
1,1773216,NaN,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel,2020-08-07,2020-08
2,2380515,NaN,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel,2021-02-15,2021-02
3,2250597,NaN,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel,2022-03-30,2022-03
4,1834446,NaN,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel,2019-09-05,2019-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431958,822590,62906.0,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase,2022-04-07,2022-04
2431959,43178,3333.0,5,e472549e-273e-402e-ae5f-38ece25699c2,2023-09-26 10:25:28+00:00,97.78.106.249,Orrell,England,WN5,IE,Email,/purchase,purchase,2023-09-26,2023-09
2431960,553230,42386.0,10,6a7ba027-2af2-46a3-ae05-294a264840cf,2023-07-31 15:11:10+00:00,147.252.137.75,Orrell,England,WN5,Chrome,Facebook,/purchase,purchase,2023-07-31,2023-07
2431961,1015190,77546.0,5,29e5960c-214b-456e-8c38-fe6eb8ff1ec8,2024-01-02 14:26:38+00:00,112.209.120.82,Wolverhampton,England,WV6,Chrome,Adwords,/purchase,purchase,2024-01-02,2024-01


In [ ]:
event.query('event_type=="purchase" and year_month <= "2022-12"')

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type,time,year_month
23068,984631,75239.0,10,43d77105-0da0-4381-b9da-edc7e7ae0045,2022-10-01 23:44:19+00:00,52.83.112.123,São Paulo,São Paulo,02675-031,Chrome,Email,/purchase,purchase,2022-10-01,2022-10
23070,1071846,81970.0,10,d0f6607c-3f4f-476f-9153-81793c782ebd,2020-11-01 10:01:37+00:00,158.127.181.83,São Paulo,São Paulo,02675-031,Safari,Adwords,/purchase,purchase,2020-11-01,2020-11
23071,1144499,87587.0,13,5ce2b5d7-5c19-407e-bbf0-af2803d7295c,2021-06-19 14:36:46+00:00,7.61.124.107,São Paulo,São Paulo,02675-031,Other,Email,/purchase,purchase,2021-06-19,2021-06
23073,824884,63068.0,7,625242f7-88ce-4479-8426-5f374af17384,2022-11-15 05:35:23+00:00,107.194.204.195,Nanyang,Beijing,100009,Safari,Facebook,/purchase,purchase,2022-11-15,2022-11
23078,419275,32139.0,5,011757ac-74a0-4d0a-999b-bc8f9fa11239,2022-08-18 06:24:59+00:00,98.57.5.191,Weihai,Beijing,100020,Chrome,Adwords,/purchase,purchase,2022-08-18,2022-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431950,445393,34115.0,7,bfa3bce6-ed81-4419-8dea-0121b3d07c1c,2022-01-23 00:31:59+00:00,103.183.246.141,Telford,England,TF2,Firefox,Email,/purchase,purchase,2022-01-23,2022-01
2431952,1227046,93890.0,13,9132d8b4-1f92-4939-8cf9-ad4f8c348fab,2022-04-01 11:00:25+00:00,27.12.96.182,Uxbridge,England,UB8,Chrome,Email,/purchase,purchase,2022-04-01,2022-04
2431957,964398,73628.0,5,959d7ba8-e279-4581-97ce-bc8ef6d754db,2022-04-07 05:50:49+00:00,176.114.208.145,Warrington,England,WA5,Chrome,Adwords,/purchase,purchase,2022-04-07,2022-04
2431958,822590,62906.0,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase,2022-04-07,2022-04


In [ ]:
event.groupby('year_month')['created_at'].count()

year_month
2019-01     18059
2019-02     17594
2019-03     19801
2019-04     19846
2019-05     20663
            ...  
2023-09     75061
2023-10     83393
2023-11     89113
2023-12    106248
2024-01    104948
Name: created_at, Length: 61, dtype: int64

In [ ]:
orders

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1
3,89,65,Cancelled,F,2020-08-13 09:58:00+00:00,NaN,NaN,NaN,1
4,102,76,Cancelled,F,2023-01-17 08:17:00+00:00,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...
125221,125196,99972,Shipped,M,2022-06-25 11:56:00+00:00,NaN,2022-06-28 08:24:00+00:00,NaN,1
125222,125199,99975,Shipped,M,2022-07-20 02:19:00+00:00,NaN,2022-07-22 11:08:00+00:00,NaN,4
125223,125208,99983,Shipped,M,2022-03-14 15:06:00+00:00,NaN,2022-03-17 05:15:00+00:00,NaN,1
125224,125210,99983,Shipped,M,2023-03-01 15:06:00+00:00,NaN,2023-03-02 10:50:00+00:00,NaN,2


In [ ]:
orders['status'].unique()

array(['Cancelled', 'Complete', 'Processing', 'Returned', 'Shipped'],
      dtype=object)

In [ ]:
orders.query('status == "Cancelled" or status == "Returned"')

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1
3,89,65,Cancelled,F,2020-08-13 09:58:00+00:00,NaN,NaN,NaN,1
4,102,76,Cancelled,F,2023-01-17 08:17:00+00:00,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...
106469,125121,99907,Returned,M,2023-11-26 00:13:00+00:00,2023-11-30 14:48:00+00:00,2023-11-27 05:34:00+00:00,2023-11-27 17:43:00+00:00,2
106470,125123,99907,Returned,M,2023-09-13 00:13:00+00:00,2023-09-19 16:28:00+00:00,2023-09-15 22:28:00+00:00,2023-09-19 06:41:00+00:00,1
106471,125129,99914,Returned,M,2023-03-07 00:41:00+00:00,2023-03-12 15:47:00+00:00,2023-03-08 21:18:00+00:00,2023-03-10 18:21:00+00:00,1
106472,125143,99925,Returned,M,2022-11-07 03:28:00+00:00,2022-11-11 19:13:00+00:00,2022-11-09 03:17:00+00:00,2022-11-09 23:26:00+00:00,1
